[Do it! BERT와 GPT로 배우는 자연어 처리](https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook) 코드를 많이 참고하고 있습니다.

#### **각종 설정하기**

In [1]:
# Library
import os
import torch
import transformers
from glob import glob
from dataclasses import dataclass, field

C:\Anaconda3\envs\torch-py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Arguments
@dataclass
class TrainArguments :
    pretrained_model_name : str = field(default="gogamza/kobart-base-v2")
    downstream_model_dir : str = field(default="./checkpoint-kobart")
    max_input_seq_length : int = field(default=32)
    max_target_seq_length : int = field(default=32)
    save_top_k : int = field(default=1)
    monitor : str = field(default = "min val_loss")
    epochs : int = field(default = 10)
    batch_size : int = field(default=16)
    learning_rate : float = field(default = 5e-5)
    force_download: bool = field(default=False)
    seed : int = field(default=None, metadata = {"help" : "random seed."})
    test_mode: bool = field(default=False,metadata={"help": "Test Mode enables `fast_dev_run`"})
    cpu_workers: int = field(default=os.cpu_count(),metadata={"help": "number of CPU workers"})
    fp16: bool = field(default=False,metadata={"help": "Enable train on FP16"})

In [3]:
args = TrainArguments(
    max_input_seq_length = 1024,
    max_target_seq_length = 128,
    batch_size = 4 if torch.cuda.is_available() else 2,
    learning_rate = 5e-3,
    epochs = 5,
    seed = 7,
)

#### **토크나이저 준비**

In [4]:
# Tokenizer
import transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
print(tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o"))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']


#### **데이터 불러오기 / 전처리**

In [5]:
import os
import csv
import time
import torch
import logging
from typing import List, Optional
from dataclasses import dataclass
from torch.utils.data.dataset import Dataset

@dataclass
class GenerationExample :
    passage : str
    summary : str

@dataclass
class GenerationFeatures : 
    input_ids : List[int]
    attention_mask : Optional[List[int]] = None
    token_type_ids : Optional[List[int]] = None
    labels : Optional[List[int]] = None

In [6]:
def data_collator(features):
    """
    Very simple data collator that:
    - simply collates batches of dict-like objects
    - Performs special handling for potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    - does not do any additional preprocessing
    i.e., Property names of the input object will be used as corresponding inputs to the model.
    See glue and ner for example of how it's useful.
    """

    # In this function we'll make the assumption that all `features` in the batch
    # have the same attributes.
    # So we will look at the first element as a proxy for what attributes exist
    # on the whole batch.
    if not isinstance(features[0], dict):
        features = [vars(f) for f in features]

    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            else:
                batch[k] = torch.tensor([f[k] for f in features], dtype=torch.long)

    return batch

In [7]:
import re
import json
from glob import glob

# Corpus
class StoryCorpus :
    def __init__(self, f_path, mode) :
        # load raw data 
        passages = []
        summaries = []
        ## 2~3sent
        for f_name in glob(os.path.join(f_path, mode ,"literature/2~3sent/*.json")) :
            with open(f_name, 'r',encoding="utf-8") as f:
                json_data = json.load(f)
                passages.append(json_data['Meta(Refine)']['passage'])
                summaries.append(json_data['Annotation']['summary1'])
        
        ## 20per
        for f_name in glob(os.path.join(f_path, mode ,"literature/2~3sent/*.json")) :
            with open(f_name, 'r',encoding="utf-8") as f:
                json_data = json.load(f)
                passages.append(json_data['Meta(Refine)']['passage'])
                summaries.append(json_data['Annotation']['summary1'])
                           
        # create examples
        passages = [re.sub("(\([^\(\)]+\))","", passage) for passage in passages]
        self.examples = self._create_examples(passages, summaries)

    def _get_examples(self) :
        return self.examples

    def _create_examples(self, passages, summaries) :
        examples = []
        for i in range(len(passages)) :
            examples.append(GenerationExample(passage=passages[i], summary=summaries[i]))
        return examples

# Convert examples to features
def _convert_examples_to_generation_features(
    examples : List[GenerationExample],
    tokenizer, args):
  
    features = []
    for example in examples :
        # tokenization
        summary_encoding = tokenizer(example.summary,
                                     max_length = args.max_target_seq_length,
                                     padding = "max_length",
                                     truncation = True,)
        with tokenizer.as_target_tokenizer() :
            passage_encoding = tokenizer(example.passage,
                                         max_length = args.max_input_seq_length,
                                         padding = "max_length",
                                         truncation = True,)
        
        # example to feature
        inputs = {k:passage_encoding[k] for k in passage_encoding}
        del(inputs['token_type_ids'])
        feature = GenerationFeatures(**inputs, labels=summary_encoding["input_ids"])
        features.append(feature)

    for i, example in enumerate(examples[:5]) :
        print("*** Example ***")
        print("summary : %s" % (example.summary))
        print("passage : %s" % (example.passage))
        print("tokens : %s" % (" ".join(tokenizer.convert_ids_to_tokens(features[i].input_ids))))
        print("features : %s" % features[i])
    return features
                                              
# Dataset
class StoryDataset(Dataset) :
    def __init__(self, examples,args, tokenizer,mode,
               convert_examples_to_features_fn = _convert_examples_to_generation_features,):
        if examples is not None : 
            self.examples = examples
        else :
            raise KeyError("examples is not valid")

        # Load data features from dataset file
        print(f"Creating features from {mode}-examples")
        tokenizer.pad_token = tokenizer.eos_token
        self.features = convert_examples_to_features_fn(examples,tokenizer,args)

    def __len__(self) :
        return len(self.features)
    
    def __getitem__(self, i) :
        return self.features[i]

    def get_labels(self):
        return self.features["labels"]

In [8]:
# train data
train_corpus = StoryCorpus("D:/jupyter/story-generation/data/story/summarization","train")
train_features = StoryDataset(train_corpus.examples, args, tokenizer,'train', _convert_examples_to_generation_features)

Creating features from train-examples
*** Example ***
summary : 아내에게 돈이 있을지 모른다는 귀띔을 문 서방은 믿을 수 없었으나 아내에게는 역시 돈이 있었다.
passage :  그러나 그보다도 절통한 것은 아내의 어리석음에서 생긴 비극이었다. 문 서방이 한이나 없게 마지막으로 굿이나 한 번 해본다고 빚을 얻으러 갈팡질 팡 다닐 때 아이 어머니한테 돈십원쯤은 있을지 모른다고 귀띔해주는 사람 이 있었다. 도랫말 술집 여편네였다. 자기도 작년에 한 번 오원을 빚내다 쓴 일이 있다는 것이었다. 처음엔 그럴듯이 들었다. 그러나 그는 그것을 믿을 수 없었다. 믿을 수는 없으면서도 아내한테 물어 보았던 것이나 역시 그런 돈이 있을 리 만무였다. 그랬던 것이 역시 아내에게는 돈이 있었다. 개가해올 때 몸에 지니고 왔던 돈 십오원에 살을 붙이고 붙이고 해서 꽁꽁 뭉친 채 1100원 돈이 있었다. 연 전 중식이란 놈이 담으로 앓을 때 친정에 가서 얻어왔다던 전후 이십원 돈 이 그의 주머니 속에서 나왔다는 것을 안 것도 그가 죽은 후였다. 전실 자 식을 살리는 데는 아낌이 없이 돈을 내놓고는 자기 병에는 일전 한 푼 못 쓰고 죽어버린 어리석은 서모. 그런 아내의 심정을 생각할 때 문 서방은 그 어리석은 서모를 위해서 다시 한 번 울어주지 않을 수 없었다. “빌어먹을 사람!” 문 서방이 장군바위 넘어 잔등을 타고 싸릿골 쪽으로 내려갈 무렵에 문득 눈이 날리기 시작하였다. 금년 접어들어 첫눈이었다. 첫눈치고는 송이가 컸 다. 문 서방은 문득 발을 멈추고 십여 년 전 본처를 묻으러 가던 날도 눈이 왔던가? 그런 생각을 해보는 것이었다. 역시 그때도 눈이 왔던 성싶었다.
그는 생각 없이 손바닥을 내벌렸다. 활짝 핀 매화송이처럼 소담스런 눈송이 가 한 개 두 개 손바닥 위에 떨어진다. 먼저 떨어진 눈송이가 녹으면 다시 파시시 한 송이 내려와 앉는다. 잠깐 동안에 그 눈송이도 사르르 녹아버린 다.

tokens : ▁그러나 ▁그 보다도 

In [9]:
# valid data
valid_corpus = StoryCorpus("D:/jupyter/story-generation/data/story/summarization","valid")
valid_features = StoryDataset(valid_corpus.examples, args, tokenizer,'valid', _convert_examples_to_generation_features)

Creating features from valid-examples
*** Example ***
summary : 동경으로 떠나게 된 실은 눈을 붉히며 떠날 용기가 나지 않아서 떠나는 날짜를 흐려오던 것이다.
passage :  “이 걱정쟁이 같으니 누굴 칠면조나 카멜레온으로 아나부다.” “저 없는 동안에 모두들 충충대서 마음을 변하게 하문 어떻게 해요. 정말 걱정예요.─전 그렇게 되면 죽을걸요 뭘.” “어서 내 염려 말구 당신 마음의 고삐나 든든히 잡아 둬요. 행여나 대중없 이 노여나지 나 말게.” “인전 그만둬요 그런 소리. 듣기만 해두 소름이 끼쳐요.” 지난 두 달 동안의 변화와 수많은 굴곡을─행복과 불행의 가지가지를 반성하면서 벌써 그것이 과거가 되고 추억이 된 것이 신기해서 견딜 수 없었다. 뭇 인물들의 왕래와 미묘한 인심까지를 아울러 생각할 때 두 사람이 꾸며 떼어 그 조그만 한 폭의 역사가 또한 인간생활의 장한 한 페이지로 여겨졌다. 그 한 폭을 주초로 하고 앞날의 발전이 훤하게 내다보이는 것이 두 사람의 마음을 한량없이 밝게 해주었다. 스스로의 운명을 스스로들 개척해 가는 용기 앞에는 하나의 확고한 결정이 있을 뿐이었다. 미래에 속하되 미래가 아닌 결정이었다. 삼한이 풀리고 사온이 시작되는 날 드디어 실은 동경으로 길을 떠나게 되었다. 날마다 학교로 오는 전화가 그날은 특별히 아침 일찍이 왔다. “오늘 떠나게 될는지두 모르겠어요. 안녕히 계셔요.” 실은 역에서 보냄을 받기를 좋아하지 않는 성질에 떠나는 날짜의 결정을 언제나 확 적히 작정하지 않고 흐려오던 것이었다. 세상에 작별같이 마음 성가신 일이 없어서 역에서 마주 보고 눈들을 붉히면 도저히 떠날 용기가 생기지 않는다는 것이었다. 언제나 떠나게 되면 말없이 가만히 떠나겠다고 하던 것을 생각하고 그날 아침 전화로 준보는 혹시 이날이 아닌가 설레면서 물었다.

tokens : ▁“이 ▁걱정 쟁이 ▁같 으니 ▁누 굴 ▁칠 면 조 나 ▁카 멜 레 온 으로 ▁아나 부 다. ” ▁“ 저 ▁없는 ▁동안 에

In [10]:
# DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_dataloader = DataLoader(train_features,
                              batch_size = args.batch_size,
                              sampler = RandomSampler(train_features, replacement=False),
                              collate_fn = data_collator,
                              drop_last = False,
                              num_workers = 0)
valid_dataloader = DataLoader(valid_features, 
                            batch_size = args.batch_size,
                            sampler = SequentialSampler(valid_features),
                            collate_fn = data_collator,
                            drop_last = True,
                            num_workers = 0,)

#### **모델 초기화**

In [11]:
# model
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained(args.pretrained_model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


#### **파인 튜닝**

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR

# Define Task Module
class StoryGenerator(pl.LightningModule) :
    def __init__(self,model, args) :
        super().__init__()
        self.model = model
        self.args = args

    def configure_optimizers(self) :
        optimizer = optim.Adam(self.parameters(), lr=self.args.learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9)
        return {'optimizer' : optimizer,
                'scheduler' : scheduler,
        }

    def training_step(self, inputs, batch_idx) :
        outputs = self.model(**inputs)
        self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
        return outputs.loss
    
    def validation_step(self, inputs, batch_idx) :
        outputs = self.model(**inputs)
        self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return outputs.loss

In [13]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

# Define Trainer function
def get_trainer(args,return_trainer_only=True) :
    ckpt_path = os.path.abspath(args.downstream_model_dir)
    os.makedirs(ckpt_path, exist_ok=True)

    # Checkpoint Callback
    checkpoint_callback = ModelCheckpoint(
      dirpath = ckpt_path, 
      filename = '{epoch}-{val_loss:.2f}',
      save_top_k = args.save_top_k,
      mode = args.monitor.split()[0], # "min"
      monitor = args.monitor.split()[1]  # "val_loss"
      )

    # Trainer
    trainer = Trainer(
        max_epochs = args.epochs,
        fast_dev_run = args.test_mode,
        num_sanity_val_steps = None if args.test_mode else 0,
        callbacks = [checkpoint_callback],
        default_root_dir = ckpt_path,

        # For GPU Setup
        deterministic=torch.cuda.is_available() and args.seed is not None,
        gpus = torch.cuda.device_count() if torch.cuda.is_available() else None,
        precision = 16 if args.fp16 else 32,
        )

    if return_trainer_only :
        return trainer
    else : 
        return checkpoint_callback, trainer

In [14]:
# Model
task = StoryGenerator(model, args)

# Fine-tuning
trainer = get_trainer(args)
trainer.fit(
    task,
    train_dataloaders = train_dataloader,
    val_dataloaders = valid_dataloader,
)

C:\Anaconda3\envs\torch-py38\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Anaconda3\envs\torch-py38\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory D:\jupyter\story-generation\checkpoint-kobart exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Anaconda3\envs\torch-py38\lib\site-packages\pytorch_lightning\core\optimizer.py:382: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'sch

Epoch 0:  89%|███████████████████████████████████████████▌     | 2400/2700 [17:02<02:07,  2.35it/s, loss=2.53, v_num=6]
Validation: 0it [00:00, ?it/s]
Epoch 1:  89%|█████████████████████████████▎   | 2400/2700 [17:45<02:13,  2.25it/s, loss=2.51, v_num=6, val_loss=2.530]
Validation: 0it [00:00, ?it/s]
Epoch 2:  89%|█████████████████████████████▎   | 2400/2700 [18:30<02:18,  2.16it/s, loss=2.43, v_num=6, val_loss=2.550]
Validation: 0it [00:00, ?it/s]
Epoch 3:  89%|█████████████████████████████▎   | 2400/2700 [19:09<02:23,  2.09it/s, loss=2.46, v_num=6, val_loss=2.460]
Validation: 0it [00:00, ?it/s]
Epoch 4:  89%|█████████████████████████████▎   | 2400/2700 [18:17<02:17,  2.19it/s, loss=2.41, v_num=6, val_loss=2.450]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|█████████████████████████████████| 2700/2700 [18:59<00:00,  2.37it/s, loss=2.41, v_num=6, val_loss=2.450]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█████████████████████████████████| 2700/2700 [18:59<00:00,  2.37it/s, loss=2.41, v_num=6, val_loss=2.450]


#### **인퍼런스**

In [18]:
text = "야릇한 방, 페페의 정성, 준비된 식탁, 갸비이의 호기심, 페페의 열정 ─ 두 사람의 사랑은 세상에서 제일가는 신기하고도 뜨거운 것이다. 갸비이의 두 눈은 별같이 탄다……. 그 불타는 화면에서 문득 내 시선을 떼게 한 것은 몇 자리 앞에 앉은 아키임과 마리이의 돌연한 거동이었다. 영화에서 감동을 받음인지 별안간 페페와 갸비이를 모방해서 그들의 열정을 연장시킨 것이다. 충동적으로 몸을 쏠리더니 번개같이 얼굴을 댄다. 어둠 속으로도 그 열광적인 자태는 또렷하게 눈에 띠었다. 그 순간 눈을 굴린 것은 나만이 아닐 듯싶다.그들은 한참이나 있다가 얼굴을 뗐으나 몸은 그대로 가까웠다. 나는 영화에서는 벌써 마음이 떠서 두 사람만을 쏘아보게 되었다. 변괴는 뒤를 이어 일어났다. 두 사람의 거동을 보고서인지 옆에 앉았던 크리이긴은 벌써 자리에서 일어섰다. 무죽거리다가 아키임들을 향해 무어라고 지껄이더니 마리이의 손을 잡는 것이었다. 함께 밖으로 나가자는 눈치인 듯했다. 아키임이 대꾸하면서 엉거주춤 자리를 일어서서 실내기를 치다가 관객의 눈을 끌 것을 두려워함인지 주저앉으니까 크리이긴도 자리에 앉았다. 앉아서도 오고 가는 말이 한참이나 많은 모양이더니 이윽고 크리이긴은 혼자 자리를 일어서서 사잇길을 지나 비틀비틀 밖으로 나가 버렸다. 남은 아키임과 마리이는 아까와는 다른 조금 불안한 듯한 기색으로 정신없이 지껄거린다. 마음을 가라앉히기에는 오랜 시간이 걸리는 눈치였다. 크리이긴은 다시 안 들어오고 두 사람은 수군거리면서 벌써 영화는 보면 말면 하는 기색이었다"
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         do_sample=True,
                         max_length=1024,
                         min_length=100,
                        top_p = 0.92)
generated = tokenizer.decode(gen_ids[0])
print(generated)

</s>과은으며다.은자 수이다.한의서적한에게하고 과었다.했다.에게 그도은에게할다.을까지에치했다.나를에 있는 수은 은었다. 수치가다.적까지다. 있는까지서할 나는적으로한가을치도나에서에다. 시다.다. 수한로히에 두했다.를의 그 의이는다. 수에게한 나는다.하여가서 문하고하고이는에치했다. 나는 </s>


In [20]:
text = '마루에서 아버지가 잠이 들어 있었는데 문득 잠든 아버지의 얼굴을 보니 센 머리털과 머리보다 센 수염이 이날따라 새삼스럽게 업순이 눈에 들어왔다.'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         do_sample=True,
                         max_length=1024,
                         min_length=10)
generated = tokenizer.decode(gen_ids[0])
print(generated)

</s>가  것이자를이에게은었다.</s>
